In [ ]:
import numpy as np
import os

from skimage import io
from tqdm import tqdm
import napari
import pandas as pd

import zarr

In [ ]:
# load information
info_path = '/mnt/ampa02_data01/tmurakami/model_training/info.pkl'
lookup_path = '/mnt/ampa02_data01/tmurakami/model_training/norm_values.pkl'
obj = pd.read_pickle(info_path)
lookup = pd.read_pickle(lookup_path)
# image path
save_path = '/mnt/ampa02_data01/tmurakami/model_training/crops'

In [ ]:
for i in tqdm(range(obj.shape[0])):
    # print(i)
    n5_path = obj.loc[i,'source']

    # create Zarr file object
    img_zarr = zarr.open(store=zarr.N5Store(n5_path), mode='r')

    corner_positions = obj.loc[i,'corner']
    crop_size = obj.loc[i,'crop_size']
    segment_chan = obj.loc[i,'channel']
    reference_chan = obj.loc[i,'ref_channel']
    plane_position = int(obj.loc[i,'plane_position'])

    # load images according to the input parameters.
    n5_setups = list(img_zarr.keys())
    img_ref = img_zarr[n5_setups[reference_chan]]['timepoint0']['s0']
    img_ref_ = img_ref[tuple(slice(i,i+j) for i,j in zip(corner_positions, crop_size))][plane_position,...]
    img_ref_norm = (img_ref_.astype(float) - lookup[n5_path][reference_chan]['lower']) / (lookup[n5_path][reference_chan]['upper'] - lookup[n5_path][reference_chan]['lower'])

    img = img_zarr[n5_setups[segment_chan]]['timepoint0']['s0']
    img_ = img[tuple(slice(i,i+j) for i,j in zip(corner_positions, crop_size))][plane_position,...]
    img_norm = (img_.astype(float) - lookup[n5_path][segment_chan]['lower']) / (lookup[n5_path][segment_chan]['upper'] - lookup[n5_path][segment_chan]['lower'])


    imgs = np.stack([img_ref_norm,img_norm])
    prefix = str(i)
    while len(prefix) < 4:
        prefix = '0' + prefix
        
        
    img_path = os.path.join(save_path, prefix+'_img_norm.tif')
    io.imsave(img_path, imgs, plugin='tifffile', metadata={'axes': 'CYX'})